<a href="https://colab.research.google.com/github/nagasora/MITSUI-CO.-Commodity-Prediction-Metric/blob/main/level1_model_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import json
f = open("/content/drive/MyDrive/kaggle notebook/kaggle.json")
json_data = json.load(f)
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']


In [3]:
#APIコマンドを入力
!kaggle competitions download -c mitsui-commodity-prediction-challenge

  0% 0.00/9.94M [00:00<?, ?B/s]
100% 9.94M/9.94M [00:00<00:00, 1.11GB/s]


In [4]:
import os
import zipfile

# Specify the path to the zip file in Google Drive
zip_file_path = "/content/mitsui-commodity-prediction-challenge.zip"

# Specify the destination directory (same as the zip file directory)
destination_directory = os.path.dirname(zip_file_path)

# Create the destination directory if it doesn't exist
os.makedirs(destination_directory, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_directory)

print(f"File unzipped to: {destination_directory}")

File unzipped to: /content


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv('/content/train.csv')
train_labels = pd.read_csv('/content/train_labels.csv')
target_pairs = pd.read_csv('/content/target_pairs.csv')

print('training data:', train.shape)
print('train label:', train_labels.shape)
print('target_pairs:', target_pairs.shape)

training data: (1917, 558)
train label: (1917, 425)
target_pairs: (424, 3)


In [7]:
# ===================================================================
# 1. セットアップとデータ読み込み
# ===================================================================
import os
import gc
import pickle
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping

# --- データパス ---
TRAIN_PATH = '/content/train.csv'
LABELS_PATH = '/content/train_labels.csv'
TARGET_PAIRS_PATH = '/content/target_pairs.csv'
OOF_PREDS_PATH = '/content/drive/MyDrive/kaggle notebook/MITSUI&CO. Commodity Prediction Challenge/oof_predictions_lgbm.csv' # ベースライン予測

# 保存用ディレクトリ
MODEL_DIR = '/content/drive/MyDrive/kaggle notebook/MITSUI&CO. Commodity Prediction Challenge/lstm_models_masked_ver1'
SCALER_DIR = '/content/drive/MyDrive/kaggle notebook/MITSUI&CO. Commodity Prediction Challenge/scalers_masked_ver1'
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(SCALER_DIR, exist_ok=True)

# --- データを読み込み、結合する ---
print("データの読み込みを開始します...")
train_df = pd.read_csv(TRAIN_PATH)
labels_df = pd.read_csv(LABELS_PATH)
target_pairs_df = pd.read_csv(TARGET_PAIRS_PATH)
featured_df = pd.merge(train_df, labels_df, on='date_id', how='left')
print("データの読み込みと結合が完了しました。")

# ===================================================================
# 2. 特徴量エンジニアリング（強化版）
# ===================================================================
print("\n特徴量エンジニアリングを開始します...")

# --- 基本的なテクニカル指標 ---
# (ユーザー様の `create_technical_features` と `create_cross_asset_features` を使用)
def create_technical_features(df, price_cols, window_sizes=[5, 10, 20]):
    """テクニカル指標を生成する"""
    features = df.copy()
    for col in price_cols:
        if col in df.columns:
            for window in window_sizes:
                features[f'{col}_MA_{window}'] = features[col].rolling(window=window, min_periods=1).mean()
                features[f'{col}_STD_{window}'] = features[col].rolling(window=window, min_periods=1).std()
            features[f'{col}_Return_1d'] = features[col].pct_change(1)
            features[f'{col}_LogReturn_1d'] = np.log(features[col] / features[col].shift(1))
            ma_20 = features[col].rolling(window=20, min_periods=1).mean()
            std_20 = features[col].rolling(window=20, min_periods=1).std()
            features[f'{col}_BB_Upper'] = ma_20 + (2 * std_20)
            features[f'{col}_BB_Lower'] = ma_20 - (2 * std_20)
    return features

def create_cross_asset_features(df, asset_groups):
    """クロスアセット特徴量を生成する"""
    features = df.copy()
    for group_name, assets in asset_groups.items():
        available_assets = [asset for asset in assets if asset in df.columns]
        if len(available_assets) >= 2:
            group_returns = df[available_assets].pct_change().rolling(5).mean()
            features[f'{group_name}_Momentum_5d'] = group_returns.mean(axis=1)
            features[f'{group_name}_Volatility_5d'] = df[available_assets].pct_change().rolling(5).std().mean(axis=1)
    return features


PRICE_COLS = [col for col in train_df.columns if '_Close' in col or '_adj_close' in col or 'FX_' in col]
featured_df = create_technical_features(featured_df, PRICE_COLS)
asset_groups = {
    'Metals': [col for col in PRICE_COLS if 'LME' in col],
    'Precious_Metals': [col for col in PRICE_COLS if 'Gold' in col or 'Silver' in col or 'Platinum' in col],
    'FX': [col for col in PRICE_COLS if 'FX' in col]
}
featured_df = create_cross_asset_features(featured_df, asset_groups)

# --- スプレッド/レシオ特徴量 ---
for _, row in target_pairs_df.iterrows():
    pair = row['pair']
    if ' - ' in pair:
        asset1, asset2 = pair.split(' - ')
        if asset1 in featured_df.columns and asset2 in featured_df.columns:
            featured_df[f'SPREAD_{asset1}_{asset2}'] = featured_df[asset1] - featured_df[asset2]
            featured_df[f'RATIO_{asset1}_{asset2}'] = featured_df[asset1] / (featured_df[asset2] + 1e-6)

# --- ラグ/移動平均乖離 特徴量 ---
lags = [1, 5, 10]
windows = [5, 10, 20]
for col in PRICE_COLS:
    for lag in lags:
        featured_df[f'{col}_lag_{lag}'] = featured_df[col].shift(lag)
    for window in windows:
        ma = featured_df[col].rolling(window=window).mean()
        featured_df[f'{col}_ma_gap_{window}'] = featured_df[col] / ma

# --- OOF予測（メタ特徴量）の読み込みと結合 ---
try:
    oof_df = pd.read_csv(OOF_PREDS_PATH, index_col=0)
    oof_df.columns = [f'target_{i}' for i in range(oof_df.shape[1])]
    oof_meta_features = oof_df.add_suffix('_oof_pred')
    featured_df_with_meta = featured_df.reset_index(drop=True).merge(
        oof_meta_features.reset_index(drop=True), left_index=True, right_index=True, how='left'
    )
    print("OOF予測をメタ特徴量として追加しました。")
except FileNotFoundError:
    print("警告: OOF予測ファイルが見つかりません。メタ特徴量なしで続行します。")
    featured_df_with_meta = featured_df.copy()

# --- 最終的な前処理 ---
# 特徴量の欠損値を埋める
feature_cols_only = [col for col in featured_df_with_meta.columns if not col.startswith('target_')]
featured_df_with_meta[feature_cols_only] = featured_df_with_meta[feature_cols_only].ffill().bfill()
featured_df_with_meta.replace([np.inf, -np.inf], 0, inplace=True) # 0で埋める方が安全な場合もある

print(f"特徴量生成完了。最終的なデータ形状: {featured_df_with_meta.shape}")
gc.collect()

データの読み込みを開始します...
データの読み込みと結合が完了しました。

特徴量エンジニアリングを開始します...
OOF予測をメタ特徴量として追加しました。
特徴量生成完了。最終的なデータ形状: (1917, 4440)


0

In [10]:
# ===================================================================
# 3. ヘルパー関数とモデル定義
# ===================================================================

def create_dataset_with_masking(df, target_col, oof_col, feature_cols, sequence_length, batch_size):
    """ターゲットのNaNを削除せず、そのままシーケンスを生成する"""
    data = df.copy()

    current_features = feature_cols.copy()
    if oof_col in data.columns:
        current_features.append(oof_col)

    X_data = data[current_features].ffill().bfill()
    y_data = data[target_col] # NaNを含むターゲット

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_data)

    dataset = tf.keras.utils.timeseries_dataset_from_array(
        X_scaled, y_data,
        sequence_length=sequence_length,
        batch_size=batch_size,
        shuffle=False
    )
    return dataset, scaler

def masked_mse(y_true, y_pred):
    """y_trueがNaNである部分を無視する平均二乗誤差（MSE）"""
    mask = tf.math.is_finite(y_true)
    y_true_masked = tf.boolean_mask(y_true, mask)
    y_pred_masked = tf.boolean_mask(y_pred, mask)
    # MeanSquaredErrorクラスのインスタンスを生成して損失を計算する
    loss_fn = tf.keras.losses.MeanSquaredError()
    return loss_fn(y_true_masked, y_pred_masked)

def build_lstm_model(input_shape):
    """LSTMモデルを構築・コンパイルする"""
    model = Sequential([
        Input(shape=input_shape),
        LSTM(units=128, return_sequences=True),
        Dropout(0.3),
        LSTM(units=64, return_sequences=False),
        Dropout(0.3),
        Dense(units=64, activation='relu'),
        Dropout(0.3),
        Dense(units=1, activation='linear')
    ])
    # ★★★ カスタム損失関数を指定 ★★★
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=masked_mse)
    return model

In [11]:
# ===================================================================
# 4. 全ターゲットの学習と保存
# ===================================================================

# --- パラメータ設定 ---
SEQUENCE_LENGTH = 30
BATCH_SIZE = 64 # データ量が増えたのでバッチサイズを少し戻しても良い
EPOCHS = 100
PATIENCE = 15

# --- ターゲットと特徴量のリスト準備 ---
target_cols = [col for col in labels_df.columns if col.startswith('target_')]
base_feature_cols = [col for col in featured_df_with_meta.columns if not col.startswith('target_') and col != 'date_id']

# --- 結果記録用のリスト ---
training_results = []

print(f"\n--- 全{len(target_cols)}ターゲットの学習を開始します ---")

for i, target_col in enumerate(target_cols):
    print(f"\n[{i+1}/{len(target_cols)}] ターゲット '{target_col}' の処理を開始...")
    gc.collect()

    try:
        oof_col = f'{target_col}_oof_pred'

        # 1. データ準備
        dataset, scaler = create_dataset_with_masking(
            featured_df_with_meta, target_col, oof_col,
            base_feature_cols, SEQUENCE_LENGTH, BATCH_SIZE
        )

        # 2. データセット分割
        DATASET_SIZE = tf.data.experimental.cardinality(dataset).numpy()
        if DATASET_SIZE < 2:
            print("  > データセットが小さすぎるためスキップします。")
            training_results.append({'target': target_col, 'val_rmse': np.nan, 'status': 'Skipped (Too small)'})
            continue

        train_size = int(0.85 * DATASET_SIZE)
        train_dataset = dataset.take(train_size).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
        val_dataset = dataset.skip(train_size).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

        # 3. モデルの構築と学習
        input_shape = train_dataset.element_spec[0].shape[1:]
        lstm_model = build_lstm_model(input_shape)
        early_stopping = EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True)

        print(f"  > 学習を開始... (Train: {train_size} batches, Val: {DATASET_SIZE - train_size} batches)")
        history = lstm_model.fit(
            train_dataset, validation_data=val_dataset,
            epochs=EPOCHS, callbacks=[early_stopping], verbose=0
        )

        # 4. 評価 (マスクを考慮)
        val_preds = lstm_model.predict(val_dataset).flatten()
        y_val = np.concatenate([y.numpy() for _, y in val_dataset])

        mask = ~np.isnan(y_val) # NaNでない部分のマスク
        if np.sum(mask) == 0:
            val_rmse = np.nan # 検証セットにラベルが一つもなかった
        else:
            val_rmse = np.sqrt(mean_squared_error(y_val[mask], val_preds[mask]))

        best_epoch = np.argmin(history.history['val_loss']) + 1
        print(f"  > 学習完了。検証RMSE: {val_rmse:.4f} (at epoch {best_epoch})")

        # 5. モデルとスケーラーの保存
        lstm_model.save(os.path.join(MODEL_DIR, f'model_{target_col}.keras'))
        with open(os.path.join(SCALER_DIR, f'scaler_{target_col}.pkl'), 'wb') as f:
            pickle.dump(scaler, f)
        print(f"  > モデルとスケーラーを保存しました。")

        training_results.append({'target': target_col, 'val_rmse': val_rmse, 'status': 'Success'})

    except Exception as e:
        print(f"!!! エラー発生: {target_col} の処理中にエラー: {e}")
        import traceback
        traceback.print_exc()
        training_results.append({'target': target_col, 'val_rmse': np.nan, 'status': f'Failed: {e}'})

# --- サマリー表示 ---
print("\n--- 全てのターゲットの処理が完了しました ---")
results_df = pd.DataFrame(training_results)
if not results_df.empty:
    print("\n学習結果サマリー:")
    print(results_df.head())
    print(f"\n成功: {results_df[results_df['status']=='Success'].shape[0]} 件")
    print(f"失敗/スキップ: {results_df[results_df['status']!='Success'].shape[0]} 件")
    avg_rmse = results_df['val_rmse'].mean()
    print(f"成功したモデルの平均検証RMSE: {avg_rmse:.4f}")
    results_df.to_csv('/content/drive/MyDrive/kaggle notebook/MITSUI&CO. Commodity Prediction Challenge/lstm_masked_training_results.csv', index=False)
    print("\n学習結果のサマリーをCSVに保存しました。")


--- 全424ターゲットの学習を開始します ---

[1/424] ターゲット 'target_0' の処理を開始...
  > 学習を開始... (Train: 25 batches, Val: 5 batches)
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


  > 学習完了。検証RMSE: 0.0106 (at epoch 41)
  > モデルとスケーラーを保存しました。

[2/424] ターゲット 'target_1' の処理を開始...
  > 学習を開始... (Train: 25 batches, Val: 5 batches)


KeyboardInterrupt: 